In [2]:
import os, os.path, sys, glob
import pandas
import datetime
import re
import argparse
import logging
import pytz
import tzwhere.tzwhere as tzwhere
import difflib
import numpy as np
import lime
import lime.lime_tabular
import sklearn.preprocessing
import scipy.stats
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt

sys.path.append('../code-air-forecast')

import forecast.convertfmt
import forecast.loadcsv
import forecast.fileio
import forecast.modelsk
import forecast.modelsnn
import forecast.modelsln
import forecast.modelfit
import forecast.evaluations

# testing Doris' evaluation code as there is some error

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


input_dir="./processed_data/"
species="o3"
saveprefix = "input*"+species+"*csv"



file_ls = forecast.fileio.get_files(input_dir, pattern=saveprefix)

print(file_ls[0:3])


auto-keras is not installed
['./processed_data/input-o3-490050007.csv', './processed_data/input-o3-530330010.csv', './processed_data/input-o3-530670005.csv']


In [5]:
output_dir="./data_hist/"

for fifn in file_ls[0:15]:
    d = forecast.fileio.load_csv(fifn)
    features = list(d.keys())
    

    #print(d['value'][0:48], d['last24h'][0:48])
    #target=['last24h']  #'value', 
    
    features.remove('index')
    features.remove('date')
    features.remove('weekday')
    features.remove('month')
    features.remove('day')
    features.remove('hour')
    
    print(len(features))

        
    fn=fifn.split("-")
    filename=output_dir+fn[2]+"-histogram-"+species+".pdf"

    pdf = PdfPages(filename)
     
    fig, axs = plt.subplots(5,4, figsize=(24, 15))
    i=0
    num_bins =30
    
    # unpack all the axes subplots
    axs = axs.ravel()

    for fi in features:  #target:
                            
        dfp_feature= d[fi].apply(pandas.Series)
        print("index", fi)

        dfp_feature.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel(fi)
        
        dfp_log= np.log10(dfp_feature) 
        dfp_log.replace([np.inf, -np.inf], np.nan, inplace=True)
        i=i+1
        #print("index", fi, d[fi])
        dfp_log.plot.hist(ax=axs[i], bins=num_bins,  fontsize=10)
        axs[i].set_ylabel("log "+fi)
        
        i=i+1
        
    pdf.savefig() 
            
    pdf.close()
    plt.close()

10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
index WindDir_deg
index windspeed_m_per_s
index RH
index value
index last24h
index lastday8havg
10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
index WindDir_deg
index windspeed_m_per_s
index RH
index value
index last24h
index lastday8havg
10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
index WindDir_deg
index windspeed_m_per_s
index RH
index value
index last24h
index lastday8havg
10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
index WindDir_deg
index windspeed_m_per_s
index RH
index value
index last24h
index lastday8havg
10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
index WindDir_deg
index windspeed_m_per_s
index RH
index value
index last24h
index lastday8havg
10
index PBL_m
index Surface_pres_Pa
index Water_vap_mixing_ratio_kg_per_kg
index T_K
inde